In [ ]:
# 导入游戏库
import gym
import pygame

In [ ]:
# 定义游戏环境
class MyWrapper(gym.Wrapper):

    def __init__(self):
        #is_slippery控制会不会滑
        env = gym.make('FrozenLake-v1',
                       render_mode='rgb_array',
                       is_slippery=False)

        super().__init__(env)
        self.env = env

    def reset(self):
        state, _ = self.env.reset()
        return state

    def step(self, action):
        state, reward, terminated, truncated, info = self.env.step(action)
        over = terminated or truncated

        #走一步扣一分,逼迫机器人尽快结束游戏
        if not over:
            reward = -1

        #掉坑扣100分，游戏它本身规则就是掉坑就停止
        if over and reward == 0:
            reward = -100

        return state, reward, over

    #打印游戏图像
    def show(self):
        from matplotlib import pyplot as plt
        plt.figure(figsize=(3, 3))
        plt.imshow(self.env.render())
        plt.show()


env = MyWrapper()

env.reset()

env.show()

In [ ]:
# 初始化action value表（Q表）
import numpy as np
Q = np.zeros((16,4))
Q

In [ ]:
from IPython import display
import random


#玩一局游戏并记录数据
def play(show=False):
    data = []
    reward_sum = 0
    # 重置所有状态
    state = env.reset()
    # 停止标志位重置
    over = False
    while not over:
        # 以下实现了ε-贪婪策略
        # 选择当前状态最大action value的动作
        action = Q[state].argmax()
        # 以 10% 的概率，随机选择一个动作action，用于探索新的动作策略
        if random.random() < 0.1:
            # 随机选一个动作
            action = env.action_space.sample()
        #  执行选择的动作 action，获取新的状态 next_state、奖励 reward 和是否结束标志 over
        next_state, reward, over = env.step(action)
        # 记录数据
        data.append((state, action, reward, next_state, over))
        # 获得奖励
        reward_sum += reward
        # 状态更新
        state = next_state
        # 如果 show 参数为 True，则调用环境的 show() 方法显示游戏画面
        if show:
            display.clear_output(wait=True)
            env.show()

    return data, reward_sum


play(True)[-1]

In [ ]:
#数据池
# 用于存储 Agent 在游戏过程中收集的经验数据，这些数据将被用于训练 Agent 的策略。
class Pool:

    def __init__(self):
        self.pool = []

    def __len__(self):
        return len(self.pool)

    def __getitem__(self, i):
        return self.pool[i]

    #更新动作池
    def update(self):
        #每次更新不少于200条新数据
        old_len = len(self.pool)
        while len(pool) - old_len < 200:
            # 调用play玩一局游戏，并将获取的游戏数据 play()[0]（游戏轨迹）追加到数据池中
            self.pool.extend(play()[0])

        #只保留最新的条10000数据
        self.pool = self.pool[-1_0000:]

    #获取一批数据样本
    def sample(self):
        return random.choice(self.pool)


pool = Pool()
pool.update()

len(pool), pool[0]

In [ ]:
#训练
def train():
    #共更新N轮数据
    for epoch in range(1000):
        pool.update()

        #每次更新数据后,训练N次
        for i in range(200):

            #随机抽一条数据
            state, action, reward, next_state, over = pool.sample()

            #Q矩阵当前估计的state下action的价值
            value = Q[state, action]

            #实际玩了之后得到的reward+下一个状态的价值*0.9
            target = reward + Q[next_state].max() * 0.9

            #value和target应该是相等的,说明Q矩阵的评估准确
            #如果有误差,则应该以target为准更新Q表,修正它的偏差
            #这就是TD误差,指评估值之间的偏差,以实际成分高的评估为准进行修正
            update = (target - value) * 0.1

            #更新Q表
            Q[state, action] += update

        if epoch % 100 == 0:
            print(epoch, len(pool), play()[-1])


train()

In [ ]:
play(True)[-1]

In [ ]:
Q